# Project 02 - Image Processing

## Thông tin sinh viên

- Họ và tên: Nguyễn Thọ Tài
- MSSV: 23127255
- Lớp: 23CLC02

## Import các thư viện liên quan

In [2]:
!python -m pip install matplotlib
!python -m pip install numpy
!python -m pip install Pillow

from PIL import Image # for read, write image
import numpy as np # for matrix compute
import matplotlib.pyplot as plt # for show image
import colorsys # for convert RGB to HSL

## Helper functions

In [3]:
# Any optional parameters beyond the required ones should be defined with default values
def read_img(img_path):
	""" Read image from img_path
	returns a 2D image (numpy array)
	"""
	return np.array(Image.open(img_path))

def show_img(img_2d):
	""" Show image
	"""
	plt.imshow(img_2d)
	plt.axis('off')
	plt.show()

def save_img(img_2d, img_path: str):
	"""	Save image to img_path
	"""
	Image.fromarray(img_2d).save(img_path)

def convert_rgb_to_hsl_loop(img_2d):
    """ Convert RGB image to HSL image
    returns a 2D image (numpy array)
    """
    norm_img = np.array(img_2d) / 255.0
    height, width, _ = norm_img.shape
    hsl_array = np.zeros_like(norm_img) # since (h,s,l) shares the same space as (r, g, b)

    for i in range(height):
        for j in range(width):
            r, g, b = norm_img[i, j]
            h, l, s = colorsys.rgb_to_hls(r, g, b)
            hsl_array[i, j] = [h, s, l]

    return hsl_array

def convert_hsl_to_rgb_loop(img_2d):
    """ Convert HSL image to RGB image
    returns a 2D image (numpy array)
    """
    height, width, _ = img_2d.shape

    rgb_array = np.zeros_like(img_2d)

    for i in range(height):
        for j in range(width):
            h, s, l = img_2d[i, j]
            r, g, b = colorsys.hls_to_rgb(h, l, s)
            rgb_array[i, j] = [r, g, b]

    rgb_array = (rgb_array * 255).astype(np.uint8)
    return Image.fromarray(rgb_array)

def convert_rgb_to_hsl(img_2d):
	""" Convert RGB image to HSL image
	returns a 2D image (numpy array)
	"""
	norm_img = np.array(img_2d) / 255.0
	rgb2hsl_vec = np.vectorize(colorsys.rgb_to_hls)
	h, l, s = rgb2hsl_vec(norm_img[..., 0], norm_img[..., 1], norm_img[..., 2]) # img[..., 0] means take all previous dim but only the 0 index of the last one. (4, 4, 3) -> (4, 3)
	return np.stack([h, s, l], axis=-1)

def convert_hsl_to_rgb(img_2d):
	""" Convert HSL image to RGB image
	returns a 2D image (numpy array)
	"""
	hsl2rgb_vec = np.vectorize(colorsys.hls_to_rgb)
	r, g, b = hsl2rgb_vec(img_2d[..., 0], img_2d[..., 1], img_2d[..., 2])
	return Image.fromarray((np.stack([r, g, b], axis=-1) * 255).astype(np.uint8))

def img_change_brightness(img_2d, brightness_factor):
	pass

def img_change_contrast(img_2d, contrast_factor):
	pass

def img_flip_horizontal(img_2d):
	return img_2d[:, ::-1]

def img_flip_vertical(img_2d):
	return img_2d[::-1, :]

def img_flip_horizontal_vertical(img_2d):
	return img_2d[::-1, ::-1]

def img_crop(img_2d, start: list[int], end: list[int]):
	height, width, _ = img_2d.shape
	x1, x2 = sorted([max(0, min(start[0], width)), max(0, min(end[0], width))])
	y1, y2 = sorted([max(0, min(start[1], height)), max(0, min(end[1], height))])

	return img_2d[y1:y2, x1:x2, :]

def img_crop_quarter_center(img_2d):
	height, width, _ = img_2d.shape
	start_height, start_width = height // 4, width // 4
	return img_crop(img_2d, [start_width, start_height], [start_width * 3, start_height * 3])

def blank_mask(img_2d):
	h, w, c = image.shape
	masked_image = np.zeros_like(image)
	if c == 4: # if RGBA image, set the mask to opaque
		masked_image[..., 3] = 255
	return masked_image

def img_circle_mask(img_2d):
	height, width, _ = img_2d.shape
	Y, X = np.mgrid[0:height, 0:width]
	cx, cy = width // 2, height // 2
	r = min(cx, cx)

	circle_mask = (X - cx) ** 2 + (Y - cy) ** 2 <= r ** 2
	mask = blank_mask(img_2d)
	mask[circle_mask] = img_2d[circle_mask]
	return mask

def img_2ellipse_mask(img_2d):
	height, width, _ = img_2d.shape
	Y, X = np.ogrid[:height, :width]
	diag = (width * width + height * height) ** 0.5 - 5

	a, b = 0.875 * diag / 2, 0.5 * diag / 2
	theta = np.deg2rad(45)

	# translate the grid coord to the center
	x = X - w // 2
	y = Y - h // 2

	def rotated_ellipse_mask(x, y, a, b, theta):
		cos_t, sin_t = np.cos(theta), np.sin(theta)
		term1 = ((x * cos_t + y * sin_t) ** 2) / a ** 2
		term2 = ((x * sin_t - y * cos_t) ** 2) / b ** 2
		return (term1 + term2) <= 1

	frame_mask = np.logical_or(rotated_ellipse_mask(x, y, a, b, theta),
							   rotated_ellipse_mask(x, y, a, b, -theta))

	mask = blank_mask(img_2d)
	masked_image[frame_mask] = image[frame_mask]


def process_image(img_2d, func=[1, 2, 3,...]):
	""" Process image with a list of functions
	func: a list of functions to apply to the image
	return processed 2D image
	"""



## Your tests

In [4]:
# YOUR CODE HERE
import numpy as np
import matplotlib.pyplot as plt

image = read_img("cat.jpg")
h, w, c = image.shape

Y, X = np.ogrid[:h, :w]
cx, cy = w // 2, h // 2
r = min(cx, cy)

circle_mask = (X - cx) ** 2 + (Y - cy) ** 2 <= r ** 2

masked_image = np.zeros_like(image)
if c == 4: # if RGBA image, set the mask to opaque
	masked_image[..., 3] = 255
masked_image[circle_mask] = image[circle_mask]

save_img(masked_image, "masked_image.png")

In [16]:
Y, X = np.ogrid[:h, :w]
diag = (w * w + h * h) ** 0.5 - 5

a, b = 0.875 * diag / 2, 0.5 * diag / 2
theta = np.deg2rad(45)

# translate the grid coord to the center
x = X - w // 2
y = Y - h // 2

def rotated_ellipse_mask(x, y, a, b, theta):
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    term1 = ((x * cos_t + y * sin_t) ** 2) / a ** 2
    term2 = ((x * sin_t - y * cos_t) ** 2) / b ** 2
    return (term1 + term2) <= 1

frame_mask = np.logical_or(rotated_ellipse_mask(x, y, a, b, theta),
						   rotated_ellipse_mask(x, y, a, b, -theta))

masked_image = np.zeros_like(image)
if c == 4: # if RGBA image, set the mask to opaque
	masked_image[..., 3] = 255
masked_image[frame_mask] = image[frame_mask]
save_img(masked_image, "masked_image3.png")

## Main FUNCTION

In [8]:
# YOUR CODE HERE